In [ ]:
!pip install confluent-Kafka>=2.3.0

In [ ]:
from confluent_kafka import Producer
import json
import time
import pandas as pd

df = pd.read_csv("/content/user_viewings.csv")
# sorting based on viewed_at ascending
df = df.sort_values(by=['viewed_at'])

In [ ]:
df.tail()

,user_id,apartment_id,viewed_at,is_wishlisted,call_to_action
2280,1429,5509022471,2024-05-29 17:13:37,n,shortlisted
497,4132,5668611785,2024-05-29 17:37:54,y,shortlisted
302,178,5668625267,2024-05-29 17:49:24,y,contact_agent
1265,4473,5664583426,2024-05-29 18:07:07,y,contact_agent
2553,2809,5659275619,2024-05-29 20:44:59,y,contact_agent


### Run for 27/06/2025

In [ ]:
json_records = df.to_dict(orient='records')

In [ ]:
conf = {
    "bootstrap.servers":"pkc-n3603.us-central1.gcp.confluent.cloud:9092",
    "security.protocol":"SASL_SSL",
    "sasl.mechanisms":"PLAIN",
    "sasl.username":"",
    "sasl.password":"",
    "session.timeout.ms":45000,
    "client.id":""
}
producer = Producer(conf)

def delivery_status(err, msg):
    if(err):
        print(f"Message delivery failed: {err}")
    else:
        print(f"Message delivered to {msg.topic()} [{msg.partition()}] at offset {msg.offset()}")

topic = 'user_viewings'

for idx, record in enumerate(json_records):
    try:
        key = (str(idx)).encode("utf-8")
        value = json.dumps(record)

        producer.produce(topic, key = key, value = value, callback=delivery_status)
        producer.poll(1)

        time.sleep(1.25)
    except Exception as e:
        print(f"Error occurred: {e}")

producer.flush()

print("Messages send to Kafka successfully")


Message delivered to user_viewings [5] at offset 0
Message delivered to user_viewings [5] at offset 1
Message delivered to user_viewings [1] at offset 0
Message delivered to user_viewings [1] at offset 1
Message delivered to user_viewings [4] at offset 0
Message delivered to user_viewings [4] at offset 1
Message delivered to user_viewings [4] at offset 2
Message delivered to user_viewings [0] at offset 0
Message delivered to user_viewings [5] at offset 2
Message delivered to user_viewings [3] at offset 0
Message delivered to user_viewings [3] at offset 1
Message delivered to user_viewings [3] at offset 2
Message delivered to user_viewings [3] at offset 3
Message delivered to user_viewings [5] at offset 3
Message delivered to user_viewings [0] at offset 1
Message delivered to user_viewings [4] at offset 3
Message delivered to user_viewings [0] at offset 2
Message delivered to user_viewings [2] at offset 0
Message delivered to user_viewings [3] at offset 4
Message delivered to user_viewi

### Run 28/06/2025

In [ ]:
import pandas as pd
import random
from datetime import datetime, timedelta

# Load data
apartment = pd.read_csv('/content/apartments.csv')

# Target: Tambah 15001 record
additional_records_needed = 15001

# Ambil list ID yang valid dari File A
valid_apart_ids = apartment['id'].tolist()

start_time = datetime.strptime('2024-05-30 00:00:00', '%Y-%m-%d %H:%M:%S')
end_time = datetime.strptime('2024-07-30 23:59:59', '%Y-%m-%d %H:%M:%S')
delta = end_time - start_time
int_delta = int(delta.total_seconds())

# Buat record baru
new_records = []
for _ in range(additional_records_needed):
    new_user_id = random.randint(1, 5000)  # user_id random
    new_apart_id = random.choice(valid_apart_ids)  # Random id dari File A
    random_second = random.randint(0, int_delta)
    new_timestamp = start_time + timedelta(seconds=random_second)
    new_wishlisted = random.choice(['n', 'y'])  # Random wishlisted
    cta_options = ['contact_agent', 'shortlisted', 'reported']
    weights = [0.35, 0.45, 0.2]
    new_cta = random.choices(cta_options, weights=weights, k=1)[0]

    new_records.append({
        'user_id': new_user_id,
        'apartment_id': new_apart_id,
        'viewed_at': new_timestamp,
        'is_wishlisted': new_wishlisted,
        'call_to_action': new_cta
    })

# Convert ke dataframe
new_data = pd.DataFrame(new_records)

In [ ]:
new_data = new_data.sort_values(by=['viewed_at'])
new_data['viewed_at'] = new_data['viewed_at'].astype(str)
new_data

,user_id,apartment_id,viewed_at,is_wishlisted,call_to_action
1560,4772,5668637952,2024-05-30 00:17:17,y,reported
6318,4169,5509086939,2024-05-30 00:24:15,n,contact_agent
5661,975,5664578623,2024-05-30 00:25:47,n,shortlisted
6844,358,5668226553,2024-05-30 00:35:54,y,shortlisted
14535,3807,5664574892,2024-05-30 00:40:10,n,contact_agent
...,...,...,...,...,...
5995,969,5668632418,2024-07-30 23:35:00,y,contact_agent
5183,4895,5668623062,2024-07-30 23:38:34,n,contact_agent
11795,4221,5509089645,2024-07-30 23:43:33,y,shortlisted
1606,2789,5668614318,2024-07-30 23:48:31,y,contact_agent


In [ ]:
new_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 15001 entries, 1560 to 13761
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   user_id         15001 non-null  int64 
 1   apartment_id    15001 non-null  int64 
 2   viewed_at       15001 non-null  object
 3   is_wishlisted   15001 non-null  object
 4   call_to_action  15001 non-null  object
dtypes: int64(2), object(3)
memory usage: 1.2+ MB


In [ ]:
json_records = new_data.to_dict(orient='records')

conf = {
    "bootstrap.servers":"pkc-n3603.us-central1.gcp.confluent.cloud:9092",
    "security.protocol":"SASL_SSL",
    "sasl.mechanisms":"PLAIN",
    "sasl.username":"",
    "sasl.password":"",
    "session.timeout.ms":45000,
    "client.id":""
}
producer = Producer(conf)

def delivery_status(err, msg):
    if(err):
        print(f"Message delivery failed: {err}")
    else:
        print(f"Message delivered to {msg.topic()} [{msg.partition()}] at offset {msg.offset()}")

topic = 'user_viewings'

offset_idx = 4999
for record in json_records:
    try:
        key = (str(offset_idx)).encode("utf-8")
        value = json.dumps(record)

        producer.produce(topic, key = key, value = value, callback=delivery_status)
        producer.poll(1)
        offset_idx += 1
        time.sleep(0.5)
    except Exception as e:
        print(f"Error occurred: {e}")

producer.flush()

print("Messages send to Kafka successfully")

Streaming output truncated to the last 5000 lines.
Message delivered to user_viewings [3] at offset 2466
Message delivered to user_viewings [3] at offset 2467
Message delivered to user_viewings [5] at offset 2525
Message delivered to user_viewings [2] at offset 2532
Message delivered to user_viewings [2] at offset 2533
Message delivered to user_viewings [0] at offset 2436
Message delivered to user_viewings [2] at offset 2534
Message delivered to user_viewings [1] at offset 2512
Message delivered to user_viewings [1] at offset 2513
Message delivered to user_viewings [2] at offset 2535
Message delivered to user_viewings [4] at offset 2530
Message delivered to user_viewings [0] at offset 2437
Message delivered to user_viewings [0] at offset 2438
Message delivered to user_viewings [5] at offset 2526
Message delivered to user_viewings [5] at offset 2527
Message delivered to user_viewings [5] at offset 2528
Message delivered to user_viewings [1] at offset 2514
Message delivered to user_viewi

In [12]:
new_data.to_csv('user_viewings_new.csv', index=False)

In [ ]:
# import math
# import pandas as pd
# import json

# df = pd.read_csv("/content/user_viewings.csv")
# # sorting based on viewed_at ascending
# df = df.sort_values(by=['viewed_at'])
# json_records = df.to_dict(orient='records')

# batch_size = math.ceil(len(json_records) / 5)

# for i in range(5):
#     start_index = i * batch_size
#     end_index = min((i + 1) * batch_size, len(json_records))
#     batch_data = json_records[start_index:end_index]

#     output_filename = f'/content/user_viewings_batch_{i+1}.json'
#     with open(output_filename, 'w') as f:
#         for record in batch_data:
#             json_record = json.dumps(record)
#             f.write(json_record + '\n')


#     print(f'Saved batch {i+1} to {output_filename}')

Saved batch 1 to /content/user_viewings_batch_1.json
Saved batch 2 to /content/user_viewings_batch_2.json
Saved batch 3 to /content/user_viewings_batch_3.json
Saved batch 4 to /content/user_viewings_batch_4.json
Saved batch 5 to /content/user_viewings_batch_5.json
